<a href="https://colab.research.google.com/github/janilles/couch/blob/master/couch_funfacts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Jimmy's Hollywood fun facts

- How many YEARS of activity have users recorded in the app?

# Libraries and connection

In [0]:
# added -q for suppressing output
!pip install -q -U pymysql

import pymysql
import pandas as pd
import altair as alt


In [0]:
def connect():
    return pymysql.connect(
        host='',
        port=,
        user='',
        passwd='',
        db='',
        autocommit=True
        )

connection = connect()

def sql_to_df(sql):
    return pd.read_sql(sql, con=connection)


# Years of activitiy in the app

## Minutes in each run
NOTE: Runs within the same week have the same number of active minutes. Refer to the app's UI. 

In [0]:
week_1 = ((60 + 90) * 7 + 60) / 60 # 18.5 minutes
week_2 = ((90 + 120) * 5 + 90) / 60 # 19 minutes
week_3 = (90 + 90 + 180 + 180 + 90 + 90 + 180) / 60 # 15 minutes
week_4 = (180 + 90 + 300 + 150 + 180 + 90 + 300) / 60 # 21.5 minutes
week_5 = 5 + 3 + 5 + 3 + 5 # 21 minutes
week_6 = 5 + 3 + 8 + 3 + 5 # 24 minutes
week_7 = 25
week_8 = 28
week_9 = 30

## Select SQL

In [0]:
select = sql_to_df("""
    SELECT
        device_id,
        week_no,
        has_reached_halfway,
        has_finished
    FROM
        c25k_run
    WHERE
        has_reached_halfway = 1
        OR
        has_finished = 1
        """)

select.head()


## Mapping minutes to weeks

In [0]:
# map minutes to week numbers
select['week_minutes'] = select['week_no'].map(
    {
        1 : week_1,
        2 : week_2,
        3 : week_3,
        4 : week_4,
        5 : week_5,
        6 : week_6,
        7 : week_7,
        8 : week_8,
        9 : week_9
})

select.head()


## Calculating minutes per run

In [0]:
# helper function
def minutes_calculator(row):
    if row['has_finished'] == 1:
        return row['week_minutes']
    else:
        return row['week_minutes'] / 2


In [0]:
# apply helper funtion to dataframe
select['active_minutes'] = select.apply(

    lambda row: minutes_calculator(row),
    axis=1
)

select.head()


## Activity expressed in years

In [0]:
active_minutes = round(select['active_minutes'].sum(), 2)

# divided by 60 for an hour / 24 for a day / 365 for a year
active_years = round(active_minutes / 60 / 24 / 365, 1)


In [0]:
print(f"{active_minutes:,} of active minutes")
print(f"{active_years:} of active years")